# Using Agents as Tools

<h3>Instead of treating agents only as conversational entities, you can wrap them as callable tools (functions). This means one agent can invoke another agent programmatically, passing structured inputs and receiving structured outputs.</h3>

## Installation

Before running this notebook, make sure you have the `agent-framework` package installed. Refer to `basic_food_agent.ipynb` for detailed installation instructions.

```bash
!pip install -U agent-framework --pre
```

In [ ]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre
# You can see the basic_food_agent.ipynb for basic agent run

## API Helper Functions

These functions interact with [TheMealDB API](https://www.themealdb.com/api.php) to fetch meal data:

- `_clean_meal_data()`: Helper function that restructures raw API response by combining ingredients and their measures into a clean, LLM-friendly format.
- `get_random_meal()`: Retrieves a random meal recipe from the database.
- `get_meal_by_name()`: Searches for a specific meal recipe by name.

These functions use the `Annotated` type hint with `Field` to provide descriptions that help the LLM understand when and how to use each tool.

In [ ]:
import requests
import json
from typing import Annotated, List, Dict, Any, Optional
from pydantic import Field

# Clean the MealDB response for the LLM
def _clean_meal_data(meal: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper function to restructure the raw meal API response into a clean, 
    LLM-friendly format by combining ingredients and measures.
    """
    if not meal:
        return {}

    # Combine ingredients and measures into a single list
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append(f"{measure.strip()} {ing.strip()}".strip())

    return {
        "id": meal.get("idMeal"),
        "name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "ingredients": ingredients,
        "tags": meal.get("strTags"),
        "youtube_link": meal.get("strYoutube")
    }

# Get random meal for today
def get_random_meal() -> str:
    """
    Retrieves a random meal recipe from the database. 
    Useful when the user wants a surprise suggestion or explicitly asks for a random recommendation.

    Returns:
        str: A JSON string containing the meal name, ingredients, and cooking instructions.
    """
    try:
        response = requests.get("https://www.themealdb.com/api/json/v1/1/random.php")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"error": "No meal found."})
            
        meal = _clean_meal_data(data["meals"][0])
        return json.dumps(meal, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch random meal: {str(e)}"})

# Here we get the meal by name
def get_meal_by_name(
    meal_name: Annotated[str, Field(description="The name of the meal to search for (e.g., 'Arrabiata', 'Burger').")]
) -> str:
    """
    Searches for a specific meal recipe by name. 
    Use this when the user asks for a specific dish or wants to know how to cook a named item.

    Args:
        meal_name: The name of the dish to search for.

    Returns:
        str: A JSON string containing a list of matching meals with their details.
    """
    try:
        # The API requires a search query parameter 's'
        response = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"status": "not_found", "message": f"No meals found with the name '{meal_name}'."})
        
        # Clean and limit results (e.g., top 3 matches to save tokens)
        results = [_clean_meal_data(m) for m in data["meals"][:3]]
        return json.dumps(results, indent=2)

    except Exception as e:
        return json.dumps({"error": f"Failed to search for meal: {str(e)}"})

## Import Dependencies

Import the required libraries and Microsoft Agent Framework components:

- `asyncio`: For async/await support
- `os`: For accessing environment variables
- `json`: For JSON parsing
- `dotenv`: For loading environment variables from `.env` file
- `ChatAgent`: The main Agent class for building conversational AI agents
- `OpenAIChatClient`: Client for LLM inference using OpenAI-compatible endpoints (OpenRouter in this case)

In [ ]:
# Import core dependencies to create the agent, for Agent Framework
import asyncio
import os
import json

from dotenv import load_dotenv, find_dotenv
# Core components for building Agent, tool-enabled agents
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

## Load Environment Variables

Load environment variables from a `.env` file in the project directory. This file should contain:
- `OPENROUTER_ENDPOINT`: The OpenRouter API endpoint URL
- `OPENROUTER_API_KEY`: Your OpenRouter API key

In [ ]:
# load environment file
load_dotenv(find_dotenv())

## Setup Chat Client

Configure the `OpenAIChatClient` to use OpenRouter API, which provides access to various LLM models including NVIDIA's Nemotron model. The client is configured with:

- `base_url`: The OpenRouter API endpoint
- `api_key`: Your API key for authentication
- `model_id`: The specific model to use (NVIDIA Nemotron 3 Nano 30B in this case)

In [ ]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("OPENROUTER_ENDPOINT"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    model_id="nvidia/nemotron-3-nano-30b-a3b:free"
)

## Define Food Agent Instructions

The `AGENT_INSTRUCTIONS` define the behavior of our Food Agent:

1. **Tool Usage**: Always use the provided tools (API functions) to answer questions
2. **Response Format**: Start with appetizing descriptions, list ingredients clearly, summarize instructions, include YouTube links if available
3. **Constraints**: Keep responses under 200 words, summarize long instructions

In [ ]:
AGENT_NAME = "FoodAgent"

AGENT_INSTRUCTIONS = """You are an expert AI Chef dedicated to helping users discover and prepare delicious meals.

CORE BEHAVIORS:
1. **Tool Usage**: You have access to a recipe database. ALWAYS use the provided tools to answer questions about recipes. Do not guess or hallucinate ingredients.
   - Use `get_meal_by_name` when the user asks for a specific dish.
   - Use `get_random_meal` when the user is undecided, asks for a suggestion, or wants a surprise.

2. **Response Format**: 
   - Start with an appetizing description of the dish.
   - List key ingredients clearly (based on the tool output).
   - Summarize the cooking instructions to be easy to follow.
   - If the tool provides a YouTube link, always include it at the end.
   - Include Tool Name used for fetching response

3. **Constraints**: 
   - Keep your response friendly but strictly under 200 words. 
   - If instructions are long, summarize the key steps to fit the word limit.
"""

## Function Middleware

Middleware allows you to intercept and modify function calls in the agent framework. The `logging_function_middleware` logs:

- The function name before execution
- The function result after execution

This is useful for debugging and understanding which tools the agent is calling.

In [ ]:
# Here we add the logging Function middleware since we are working with functions
from agent_framework import AgentRunContext, FunctionInvocationContext
from typing import Callable , Awaitable

# Function middleware 
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"Calling function: {context.function.name}")

    await next(context)

    print(f"Function result: {context.result}")

## Create the Food Agent

Create the first agent (`food_agent`) with:

- `name`: "FoodAgent"
- `chat_client`: The OpenAI chat client configured earlier
- `instructions`: The behavior instructions defined above
- `tools`: The API helper functions (`get_random_meal`, `get_meal_by_name`)
- `middleware`: The logging middleware for debugging

In [ ]:
# Here we create the foodAgent
# create the agent remember we are not using any tools here, this is simple example
food_agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal, get_meal_by_name],
    middleware=[logging_function_middleware]
)

## Create Main Agent with Food Agent as Tool

**This is the key concept of this example - using an agent as a tool!**

The `main_agent` is created using `openai_chat_client.as_agent()` and includes the `food_agent` as a tool via `food_agent.as_tool()`. This demonstrates:

- **Agent as Tool**: The `food_agent` is wrapped as a callable tool that can be invoked by other agents
- **Chaining Agents**: The main agent can delegate tasks to specialized sub-agents (like the food agent)
- **Modularity**: You can have multiple specialized agents, each with their own tools, and use them as building blocks

The main agent is instructed to respond in Hindi, showing how you can control the output language even when using a sub-agent.

In [ ]:
# now we create the main agent and provide the food_agent as a tool
# Tip - We can have multiple agents which can act a tool where each agent uses multiple tools
main_agent = openai_chat_client.as_agent(
    instructions="Your are a helpful assistant who responds in Hindi.",
    tools=food_agent.as_tool()
)

## Run the Agent

Execute the main agent with a query about making Arrabiata. The agent will:

1. Receive the query in English
2. Call the `food_agent` tool (which in turn calls the `get_meal_by_name` API function)
3. Get the recipe data from the API
4. Translate and format the response in Hindi as instructed

In [ ]:
result = await main_agent.run("How to make Arrabiata?")

## Display the Result

Print the agent's response. The output should be in Hindi and contain the Arrabiata recipe information retrieved by the food agent.

In [ ]:
print(result.text)

## Summary

In this example, we demonstrated:

1. **Creating specialized agents** with their own tools and instructions
2. **Wrapping agents as tools** using the `.as_tool()` method
3. **Agent composition** - passing one agent as a tool to another agent
4. **Delegation** - the main agent can delegate tasks to specialized sub-agents

This pattern is powerful for building complex multi-agent systems where each agent has a specific role and can be composed together to solve more complex problems.